In [3]:

# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys
#
import io
import TNCmodem as TNC
import compression
import zipfile
import imageio
#
from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate
import threading

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray
from  scipy.io.wavfile import read as wavread
import newax25 as ax25

import multiprocessing

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign

In [4]:
from PIL import Image

In [5]:
# function to compute least common multipler
def lcm(numbers):
    return reduce(lambda x, y: (x*y)//gcd(x,y), numbers, 1)

In [6]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 USB Audio Device: - (hw:1,0), ALSA (1 in, 2 out)
  3 sysdefault, ALSA (0 in, 128 out)
  4 dmix, ALSA (0 in, 2 out)
* 5 default, ALSA (1 in, 2 out)

In [7]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1

In [8]:
#initialize GPIO pins
GPIO.setmode(GPIO.BOARD)

PTT = 12
GPIO.setup(PTT, GPIO.OUT, initial=0)


In [9]:
# def downsample(img, factor):
#     """Takes in an Image object and int factor. It then downsamples and returns the downsampled image"""
#     np_img = np.array(img)
#     down = np_img[::factor,::factor,:]
#     down_img = Image.fromarray(down)
#     print(down.shape)
#     return down_img

In [11]:
img_name = "createrLake.tiff"
image =  imageio.imread(img_name)
#cmp_img = ###FUNCTION###
y,cr,cb = compression.compress(image,25)
with zipfile.ZipFile("img.zip", 'w') as myzip:
    myzip.write("imagey.txt")
    myzip.write("imagecr.txt")
    myzip.write("imagecb.txt")
#add downsampled image to a byte buffer
# byte_img = io.BytesIO()
# cmp_img.save(byte_img, format='PNG', optimize=True) #converts to lossless PNG with compression
# byte_img.seek(0)
# byte_img = io.BufferedReader(byte_img)

/usr/local/lib/python3.5/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.5/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.5/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.5/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))


In [ ]:
### APRS packet send image
callsign = "KM6BKP"
f = open("img.zip","rb")

fs = 48000
modem = TNC.TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 1)
gain = 0.15


Qout = Queue.Queue()

starttime = time.time()

def send_packets():
    print("Sending packets")
    GPIO.output(PTT, GPIO.HIGH)
    npp = 0
    tmp = modem.modulatPacket(callsign, b"", 'BEGIN', b"img.zip" , preflags=80, postflags=10 )
    sd.play(gain*tmp,samplerate=fs,blocking=True)
    GPIO.output(PTT, GPIO.LOW)
    while(1):
        GPIO.output(PTT, GPIO.HIGH)
        bytes = f.read(256)
        tmp = modem.modulatPacket(callsign, b"", str(npp), bytes, preflags=20, postflags=10 )    
        sd.play(gain*tmp,samplerate=fs,blocking=True)
        GPIO.output(PTT, GPIO.LOW)

        npp = npp+1
        if len(bytes) < 256:
            break
            
    GPIO.output(PTT, GPIO.HIGH)
    tmp = modem.modulatPacket(callsign, b"", "END", b"This is the end of transmission", preflags=2, postflags=80 )
    sd.play(gain*tmp,samplerate=fs,blocking=True)
    GPIO.output(PTT, GPIO.LOW)


    print("Done")
send_packets()
print(time.time() - starttime)
byte_img.close()

Sending packets
